In [ ]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from natsort import natsorted
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc, f1_score, roc_auc_score

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Dropout, Lambda, Add, Multiply

In [ ]:
# Patch dimension details
Img_Width = 48
Img_Height = 48
Img_Channels = 1

In [ ]:
# Function for Sensitivity & Specificity

def ce_loss(y_true, y_pred):
    term_0 = (1 - y_true) * K.log(1 - y_pred + K.epsilon())
    term_1 = y_true * K.log(y_pred + K.epsilon())
    out = -K.mean(term_0 + term_1)                               # This is Binary Cross Entropy Loss
    return out

def acc_met(y_true,y_pred):
    out = K.mean(K.equal(y_true, y_pred))
    return out

def sensitivity(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    neg_y_pred = 1 - y_pred_f
    tp = K.sum(y_true_f * y_pred_f)
    fn = K.sum(y_true_f * neg_y_pred)
    return tp / (tp+fn+K.epsilon())

def specificity(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    neg_y_true = 1 - y_true_f
    neg_y_pred = 1 - y_pred_f
    fp = K.sum(neg_y_true * y_pred_f)
    tn = K.sum(neg_y_true * neg_y_pred)
    return tn / (tn + fp + K.epsilon())

In [ ]:
# Loading the model ..
seg_model = load_model("/content/UNet_JND_EOphtha.h5", custom_objects={"ce_loss":ce_loss, "acc_met":acc_met, "sensitivity": sensitivity, "specificity": specificity})

In [ ]:
######################### Testing the model for single image #########################

In [ ]:
# Loading the image and applying CLAHE

img = cv2.imread('EOphtha_43.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# The initial processing of the image
img_g = img[:,:,1]

# Applying CLAHE as pre-processing step
clahe = cv2.createCLAHE(clipLimit = 8, tileGridSize=(8,8))
img_c = clahe.apply(img_g)

In [ ]:
# loading the ground truth

[m,n] = img_c.shape

gt = cv2.imread('EOphtha_47.png',0)
gt = gt / 255.0
gt = (gt > 0.5)

In [ ]:
tot = 0
step = 8
for i in range(0,m,step):
    for j in range(0,n,step):
        if((i+48)>(m-1) or (j+48)>(n-1)):
            pass
        else:
            tot = tot + 1

In [ ]:
threshold = 0.30 # change this to high value if false positives are coming
cnt1 = 0
cnt2 = 0
iarr = []
jarr = []
patches_gat = np.zeros((tot,48,48,1))
final_res = np.zeros((m,n))

for i in range(0,m,step):
    for j in range(0,n,step):
        if((i+48)>(m-1) or (j+48)>(n-1)):
            pass
        else:
            patch_img = img_c[i:i+48,j:j+48]
            patch_img = np.expand_dims(patch_img, axis=-1)
            patch_img = np.expand_dims(patch_img, axis=0)
            patches_gat[cnt1] = patch_img
            cnt1 = cnt1 + 1
            iarr.append(i)
            jarr.append(j)

inter_res = seg_model.predict(patches_gat,verbose=False)
inter_res = (inter_res > threshold)
for k in range(cnt1):
    final_res[iarr[k]:(iarr[k]+48),jarr[k]:(jarr[k]+48)] = final_res[iarr[k]:iarr[k]+48,jarr[k]:jarr[k]+48] + np.squeeze(inter_res[cnt2])
    cnt2 = cnt2 + 1

In [ ]:
psm_th2 = final_res / np.max(final_res)
psm_th2 = (psm_th2 > 0.05).astype(float)

In [ ]:
fig = plt.figure()
fig.set_size_inches(25,25)

ax1 = fig.add_subplot(121)
ax1.imshow(img_c, cmap='gray')
ax1.set_title("Input Fundus Image")
ax1.axis(False)
ax1.grid(False)

ax2 = fig.add_subplot(122)
ax2.imshow(psm_th2, cmap="gray")
ax2.set_title("Predicted Segmented Map")
ax2.axis(False)
ax2.grid(False)

fig.tight_layout()
plt.show()